<a href="https://colab.research.google.com/github/SAHIL9581/w2w/blob/main/W2W_WNB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#@title 1. Setup Environment & Install Libraries

# --- 1. Install All Required Libraries ---
print("--> Installing all necessary Python libraries (this may take a few minutes)...")
!pip install wandb torch torchvision torchaudio lasio scikit-learn pandas tqdm matplotlib joblib pyyaml -q
print("✅ Library installation complete.")


# --- 2. Define and Change to Project Directory ---
import os

# IMPORTANT: This folder is TEMPORARY. All local files will be DELETED when the Colab session ends.
# Your results and models will be saved to your online W&B account.
PROJECT_PATH = '/content/W2W_Pipeline_WandB'

print(f"\n--> Setting up a temporary project directory at: {PROJECT_PATH}")
os.makedirs(f"{PROJECT_PATH}/data/raw_las_files", exist_ok=True)
os.makedirs(f"{PROJECT_PATH}/artifacts", exist_ok=True)
os.makedirs(f"{PROJECT_PATH}/trained_models/autoencoder", exist_ok=True)
os.makedirs(f"{PROJECT_PATH}/trained_models/boundary_detector", exist_ok=True)

# Change the current working directory to the project path
os.chdir(PROJECT_PATH)
print(f"✅ Current directory changed to: {os.getcwd()}")
print("\n--- Setup Complete ---")

--> Installing all necessary Python libraries (this may take a few minutes)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 4.5 MB/s eta 0:00:00
✅ Library installation complete.

--> Setting

In [3]:
#@title 2. Login to Weights & Biases
import wandb

print("--> ACTION REQUIRED: Please log in to your Weights & Biases account.")
# You will be prompted to paste your W&B API key.
# You can find your key here: https://wandb.ai/authorize
!wandb login

--> ACTION REQUIRED: Please log in to your Weights & Biases account.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sahilpareek203 (sahilpareek203-amrita-vishwa-vidyapeetham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:
#@title 3. Upload ZIP File with .las Data
from google.colab import files
import os

print(">>> ACTION REQUIRED: Please upload the ZIP file containing your .las files.")
uploaded = files.upload()

if not uploaded:
    print("\n⚠️ Upload was cancelled or failed. Please run this cell again.")
else:
    zip_filename = list(uploaded.keys())[0]
    print(f"\n✅ '{zip_filename}' uploaded successfully.")

    # Unzip into the designated raw data folder
    !unzip -q -o "{zip_filename}" -d data/raw_las_files/

    print("--> ZIP file has been unzipped into 'data/raw_las_files/'.")

    # Clean up the uploaded zip file from the root directory
    os.remove(zip_filename)
    print("\n✅ Data upload is complete. You can now proceed to the next step.")

>>> ACTION REQUIRED: Please upload the ZIP file containing your .las files.


Saving train.zip to train.zip

✅ 'train.zip' uploaded successfully.
--> ZIP file has been unzipped into 'data/raw_las_files/'.

✅ Data upload is complete. You can now proceed to the next step.


In [1]:
#@title 4. Pipeline Configuration (Updated with 200 Epochs)
# All settings for the pipeline are controlled from this Python dictionary.

config = {
    "run_data_preparation": True,
    "run_pretraining": True,
    "run_finetuning": True,
    "run_inference": True,

    "paths": {
        "raw_las_folder": "data/raw_las_files/",
        "processed_csv_path": "data/train.csv",
        "label_encoder_path": "artifacts/label_encoder.json",
        "std_scaler_path": "artifacts/StandardScaler.bin",
        "pretrained_encoder_path": "trained_models/autoencoder/best_autoencoder.pt",
        "final_model_path": "trained_models/boundary_detector/final_model.pt"
    },

    "wandb": {
        "project": "W2W_Matcher_Pipeline_Notebook", # Your W&B project name
        "entity": None,                             # Your W&B username or team name (optional)
        "sweep_count": 5                            # Number of hyperparameter combinations to try
    },

    "pretraining_sweep": {
        "name": "Autoencoder-Pre-training-Sweep",
        "method": "random",
        "metric": {"name": "loss", "goal": "minimize"},
        "parameters": {
            "epochs": {"value": 25}, # Pre-training epochs remain the same
            "optimizer": {"values": ["RMSprop", "AdamW", "Adam"]},
            "lr": {"values": [0.001, 0.0001]},
            "act_name": {"values": ["prelu", "relu"]},
            "batch_size": {"values": [16, 32]},
        }
    },

    "finetuning": {
        "learning_rate": 0.0001,
        "batch_size": 16,
        "epochs": 200, # <-- CHANGED: Updated from 100 to 200 as requested
        "model_params": {
            "patch_height": 700, "act_name": "prelu",
            "hidden_dim": 256, "num_queries": 100,
            "num_heads": 8, "dropout": 0.1,
            "num_transformers": 6, "output_size": 3
        },
        "matcher_costs": {"set_cost_class": 1, "set_cost_bbox": 5},
        "loss_weights": {"loss_matching": 1.0, "loss_unmatching": 0.5, "loss_height_constraint": 0.5}
    },

    "inference": {
        "reference_well": "15_9-13 Sleipner East Appr",
        "well_of_interest": "16/1-2  Ivar Aasen Appr",
        "correlation_threshold": 0.7
    }
}

print("✅ Configuration dictionary updated with 200 fine-tuning epochs.")

✅ Configuration dictionary updated with 200 fine-tuning epochs.


In [5]:
#@title 5. Define Data Preparation Function (Corrected)
import pandas as pd
import numpy as np
import lasio
import json
from sklearn.preprocessing import StandardScaler
from joblib import dump

def run_data_preparation(config):
    print("--- LAUNCHING PIPELINE 0: DATA PREPARATION ---")
    paths = config['paths']
    search_folder = paths['raw_las_folder']
    all_wells_df, las_files_found = [], []

    print(f"--> Searching for .las files in '{search_folder}'...")
    for root, dirs, files in os.walk(search_folder):
        for file in files:
            if file.lower().endswith('.las'):
                las_files_found.append(os.path.join(root, file))

    if not las_files_found: raise FileNotFoundError(f"No .las files found in '{search_folder}'.")
    print(f"--> Found {len(las_files_found)} .las files. Reading now...")

    for filepath in las_files_found:
        try:
            las = lasio.read(filepath)
            df = las.df().reset_index()
            df['WELL'] = las.well.WELL.value or os.path.splitext(os.path.basename(filepath))[0]
            df['GROUP'] = 'UNKNOWN'
            for param in las.params:
                if 'GROUP' in param.mnemonic.upper(): df['GROUP'] = param.value
            all_wells_df.append(df)
        except Exception as e: print(f"    - Could not read {filepath}: {e}")

    if not all_wells_df: raise ValueError("Could not process any .las files.")
    master_df = pd.concat(all_wells_df, ignore_index=True)
    if 'DEPT' in master_df.columns: master_df.rename(columns={'DEPT': 'DEPTH_MD'}, inplace=True)
    master_df.to_csv(paths['processed_csv_path'], index=False, sep=';')
    print(f"--> Saved combined data to '{paths['processed_csv_path']}'")

    unique_wells = master_df['WELL'].unique()
    print("\n--- Available Well Names for Inference ---")
    for well in unique_wells: print(f"- {well}")
    print("------------------------------------------")
    print("TIP: Copy/paste two of these names into the 'inference' section of the config cell above.\n")

    label_encoder = {str(g): i for i, g in enumerate(master_df['GROUP'].unique())}
    with open(paths['label_encoder_path'], 'w') as f: json.dump(label_encoder, f, indent=4)
    print(f"--> Saved label encoder to '{paths['label_encoder_path']}'")

    cols_to_drop = ['WELL', 'GROUP'] + [col for col in master_df.columns if 'DEPT' in col.upper()]
    numeric_df = master_df.drop(columns=cols_to_drop, errors='ignore').fillna(0)
    scaler = StandardScaler().fit(numeric_df)
    dump(scaler, paths['std_scaler_path'])
    print(f"--> Saved StandardScaler to '{paths['std_scaler_path']}'")

    # --- CRUCIAL FIX: Dynamically add the number of features to the config ---
    num_features = scaler.n_features_in_
    print(f"\n✅ Automatically detected {num_features} features (input channels) from the data.")
    config['finetuning']['model_params']['in_channels'] = num_features
    config['pretraining_sweep']['parameters']['in_channels'] = {'value': num_features}
    # --- END OF FIX ---

print("✅ Data preparation function defined.")

✅ Data preparation function defined.


In [6]:
#@title 6. Define Dataset Classes (Corrected)
import torch
from torch.utils import data
from joblib import load
import pandas as pd
import numpy as np

# --- CORRECTED: AutoencoderDataset now creates patches ---
class AutoencoderDataset(data.Dataset):
    def __init__(self, c):
        p = c['paths']
        # Use the same patch height as the fine-tuning stage for consistency
        patch_height = c['finetuning']['model_params']['patch_height']

        df = pd.read_csv(p['processed_csv_path'], delimiter=';')
        scaler = load(p['std_scaler_path'])

        self.data_patches = []
        # Group by each well to create contiguous patches
        for well_name, well_df in df.groupby('WELL'):
            cols_to_drop = ['WELL', 'GROUP'] + [col for col in well_df.columns if 'DEPT' in col.upper()]
            well_numeric = well_df.drop(columns=cols_to_drop, errors='ignore').fillna(0)

            # Ensure the scaler is applied with the correct feature names
            if hasattr(scaler, 'feature_names_in_'):
                well_numeric = well_numeric[scaler.feature_names_in_]

            scaled_data = scaler.transform(well_numeric).astype(np.float32)

            # Create patches from this well's data
            for i in range(0, len(scaled_data) - patch_height + 1, patch_height):
                patch = scaled_data[i:i + patch_height]
                # The input to Conv1d should be (channels, length)
                self.data_patches.append(patch.T)

    def __len__(self):
        return len(self.data_patches)

    def __getitem__(self, i):
        patch = self.data_patches[i]
        return torch.from_numpy(patch), torch.from_numpy(patch)

class BoundaryDataset(data.Dataset):
    def __init__(self, c, seed=None):
        self.p, self.d = c['finetuning']['model_params'], c['paths']
        self.s = seed or np.random.randint(2**32 - 1)
        self.x, self.gt = self.get_Xy()

    def get_Xy(self):
        d = pd.read_csv(self.d['processed_csv_path'], delimiter=';')
        np.random.seed(self.s)
        w = d[d['WELL'] == np.random.choice(d.WELL.unique())].copy()

        with open(self.d['label_encoder_path']) as f: le = json.load(f)
        w['GROUP'] = w['GROUP'].astype(str).map(le).bfill().ffill()

        cols_to_drop = ['WELL', 'GROUP'] + [col for col in w.columns if 'DEPT' in col.upper()]
        w_numeric = w.drop(columns=cols_to_drop, errors='ignore').fillna(0)

        scaler = load(self.d['std_scaler_path'])
        if hasattr(scaler, 'feature_names_in_'):
            w_numeric = w_numeric[scaler.feature_names_in_]

        s_d = scaler.transform(w_numeric)

        ph = self.p['patch_height']
        idx = list(range(0, s_d.shape[0], ph))
        x = np.asarray([s_d[i:i + ph] for i in idx if len(s_d[i:i + ph]) == ph], dtype=np.float32)
        y = np.asarray([w['GROUP'].values[i:i + ph] for i in idx if len(w['GROUP'].values[i:i + ph]) == ph])
        return x, self._get_gt_boundaries(y)

    def _get_gt_boundaries(self, y_patches):
        gts = []
        for y in y_patches:
            gt, c = {}, 0
            boundaries = np.where(y[:-1] != y[1:])[0] + 1
            k = np.concatenate(([0], boundaries, [len(y)]))
            for i in range(len(k) - 1):
                top, bottom = k[i], k[i+1]
                gt[c] = {'Group': int(y[top]), 'Top': top, 'Height': bottom - top}; c += 1
            gts.append(gt)
        return gts

    def __len__(self): return len(self.x)
    def __getitem__(self, idx):
        img = np.expand_dims(self.x[idx], 0)
        data = self.gt[idx]
        ph = self.p['patch_height']
        tops = torch.tensor([d['Top'] / ph for d in data.values()], dtype=torch.float32).view(-1, 1)
        heights = torch.tensor([d['Height'] / ph for d in data.values()], dtype=torch.float32).view(-1, 1)
        tgt = {'labels': torch.ones(len(data), dtype=torch.long), 'loc_info': torch.hstack((tops, heights))}
        return torch.from_numpy(img), tgt

print("✅ Dataset classes corrected for patching.")

✅ Dataset classes corrected for patching.


In [7]:
#@title 7. Define Model Architectures (Final Corrected Version)
import torch
import torch.nn as nn
import torch.nn.functional as F

def get_activation(name):
    """Returns the activation function based on the provided name."""
    return nn.PReLU() if name == 'prelu' else nn.ReLU() if name == 'relu' else nn.GELU()

class Block1D(nn.Module):
    """A basic 1D convolutional block with two convolution layers."""
    def __init__(self, in_channels, out_channels, stride=2, kernel_size=3, activation='prelu'):
        super().__init__()
        self.b = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size, stride, padding=kernel_size//2),
            nn.BatchNorm1d(out_channels), get_activation(activation),
            nn.Conv1d(out_channels, out_channels, kernel_size, 1, padding=kernel_size//2),
            nn.BatchNorm1d(out_channels), get_activation(activation)
        )
    def forward(self, x): return self.b(x)

class UNet1D(nn.Module):
    """A 1D U-Net architecture with robust skip connections."""
    def __init__(self, in_channels, activation='prelu'):
        super().__init__()
        self.start = Block1D(in_channels, 32, stride=1, activation=activation)
        self.e1 = Block1D(32, 64, stride=2, activation=activation)
        self.e2 = Block1D(64, 128, stride=2, activation=activation)
        self.e3 = Block1D(128, 256, stride=2, activation=activation)
        self.mid = Block1D(256, 512, stride=2, activation=activation)
        self.uc3 = nn.ConvTranspose1d(512, 256, 2, 2)
        self.d3 = Block1D(512, 256, stride=1, activation=activation)
        self.uc2 = nn.ConvTranspose1d(256, 128, 2, 2)
        self.d2 = Block1D(256, 128, stride=1, activation=activation)
        self.uc1 = nn.ConvTranspose1d(128, 64, 2, 2)
        self.d1 = Block1D(128, 64, stride=1, activation=activation)
        self.uc0 = nn.ConvTranspose1d(64, 32, 2, 2)
        self.d0 = Block1D(64, 32, stride=1, activation=activation)
        self.out_conv = nn.Conv1d(32, in_channels, 1)

    def forward(self, x):
        s1 = self.start(x); s2 = self.e1(s1); s3 = self.e2(s2); s4 = self.e3(s3); m = self.mid(s4)
        d3 = self.d3(torch.cat((F.interpolate(self.uc3(m), size=s4.shape[2]), s4), 1))
        d2 = self.d2(torch.cat((F.interpolate(self.uc2(d3), size=s3.shape[2]), s3), 1))
        d1 = self.d1(torch.cat((F.interpolate(self.uc1(d2), size=s2.shape[2]), s2), 1))
        d0 = self.d0(torch.cat((F.interpolate(self.uc0(d1), size=s1.shape[2]), s1), 1))
        return self.out_conv(d0)

class UNetEncoder1D(nn.Module):
    """The encoder part of the 1D U-Net."""
    def __init__(self, in_channels, activation='prelu'):
        super().__init__()
        self.start = Block1D(in_channels, 32, stride=1, activation=activation)
        self.e1 = Block1D(32, 64, stride=2, activation=activation)
        self.e2 = Block1D(64, 128, stride=2, activation=activation)
        self.e3 = Block1D(128, 256, stride=2, activation=activation)
        self.mid = Block1D(256, 512, stride=2, activation=activation)
    def forward(self, x):
        x = x.squeeze(1).permute(0, 2, 1); s1 = self.start(x); s2 = self.e1(s1); s3 = self.e2(s2); s4 = self.e3(s3); return self.mid(s4)

class Project(nn.Module):
    """Projects flattened features to a different dimension."""
    def __init__(self, i, o): super().__init__(); self.l = nn.Linear(i, o)
    def forward(self, x): return self.l(x.flatten(1))

class Query(nn.Module):
    """Creates a learnable query tensor for the transformer."""
    def __init__(self, s, d): super().__init__(); self.q = nn.Parameter(torch.randn(1, s, d))
    def forward(self, x): return self.q.repeat(x.shape[0], 1, 1)

class Transformer(nn.Module):
    """A standard transformer encoder layer."""
    def __init__(self, i, n, d): super().__init__(); self.t = nn.TransformerEncoderLayer(d_model=i, nhead=n, dropout=d, batch_first=True, dim_feedforward=i * 4)
    def forward(self, q, c): return self.t(q)

class W2WTransformerModel(nn.Module):
    """A transformer-based model using a U-Net encoder."""
    def __init__(self, c):
        super().__init__()
        p = c['finetuning']['model_params']
        self.encoder = UNetEncoder1D(p['in_channels'], p['act_name'])

        # --- CRUCIAL FIX: Dynamically calculate the flattened feature size ---
        with torch.no_grad():
            dummy_input = torch.randn(1, 1, p['patch_height'], p['in_channels'])
            dummy_output = self.encoder(dummy_input)
            p_in = dummy_output.flatten(1).shape[1]
            print(f"--> Dynamically calculated transformer input features: {p_in}")
        # --- END OF FIX ---

        self.project = Project(p_in, p['hidden_dim'])
        self.query = Query(p['num_queries'], p['hidden_dim'])
        self.transformers = nn.ModuleList([Transformer(p['hidden_dim'], p['num_heads'], p['dropout']) for _ in range(p['num_transformers'])])
        self.finalize = nn.Sequential(nn.Linear(p['hidden_dim'], p['output_size']), get_activation(p['act_name']), nn.LayerNorm(p['output_size']))

    def forward(self, img):
        encoded_output = self.encoder(img)
        projected_seq = self.project(encoded_output).unsqueeze(1)
        q = self.query(projected_seq)
        for t in self.transformers:
            q = t(q, projected_seq)
        return self.finalize(q)

print("✅ Model architectures defined, made robust, and fully symmetrical.")

✅ Model architectures defined, made robust, and fully symmetrical.


In [8]:
#@title 8. Define Matcher and Loss Functions (Final Corrected Version)
from scipy.optimize import linear_sum_assignment
import torch
import torch.nn as nn
import torch.nn.functional as F

class HungarianMatcher(nn.Module):
    """
    This class computes an assignment between the model's predictions and the ground truth.
    It loops through each sample in the batch to create a 2D cost matrix, which is what
    the assignment algorithm expects.
    """
    def __init__(self, cost_class: float = 1, cost_bbox: float = 1):
        super().__init__()
        self.cost_class = cost_class
        self.cost_bbox = cost_bbox

    @torch.no_grad()
    def forward(self, outputs, targets):
        bs, num_queries = outputs.shape[:2]

        indices = []
        # --- CRUCIAL FIX: Iterate over each sample in the batch ---
        for i in range(bs):
            out_prob = outputs[i, :, :1].sigmoid() # Probabilities for this sample
            out_bbox = outputs[i, :, 1:]           # Predicted boxes for this sample

            tgt_bbox = targets[i]["loc_info"].to(out_prob.device) # Target boxes for this sample

            # Compute the classification cost (L1)
            cost_class = -out_prob

            # Compute the L1 cost between boxes
            cost_bbox = torch.cdist(out_bbox, tgt_bbox, p=1)

            # Final cost matrix for this sample (shape: [num_queries, num_targets])
            C = self.cost_bbox * cost_bbox + self.cost_class * cost_class

            # Run the assignment algorithm on the 2D cost matrix
            indices.append(linear_sum_assignment(C.cpu()))

        return [(torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]


class SetCriterion(nn.Module):
    def __init__(self,c):
        super().__init__(); p=c['finetuning']; self.m=HungarianMatcher(p['matcher_costs']['set_cost_class'],p['matcher_costs']['set_cost_bbox']); self.w=p['loss_weights']; self.nq=p['model_params']['num_queries']
    def loss_match(self,o,t,i):
        src_idx = self._get_src_p_idx(i)
        target_boxes = torch.cat([v["loc_info"][j] for v,(_,j) in zip(t,i) if len(j)>0], dim=0)
        if target_boxes.numel() == 0: return {'loss_matching': torch.tensor(0.0, device=o.device)}

        pred_boxes = o[src_idx]
        return {'loss_matching': F.l1_loss(pred_boxes[:, 1:], target_boxes)}

    def loss_unmatch(self,o,t,i):
        # Create a mask for matched predictions
        src_idx = self._get_src_p_idx(i)
        mask = torch.ones(o.shape[0], o.shape[1], dtype=torch.bool, device=o.device)
        mask[src_idx] = False

        # Get the scores of unmatched predictions
        unmatched_scores = o[mask][:, 0].sigmoid()
        return {'loss_unmatching': unmatched_scores.mean()}

    def loss_height(self,o,t,i):
        src_idx = self._get_src_p_idx(i)
        if src_idx[0].numel() == 0: return {'loss_height_constraint': torch.tensor(0.0, device=o.device)}

        pred_heights = o[src_idx][:, 2]
        batch_indices = src_idx[0]

        # Calculate the sum of heights for each sample in the batch
        total_height_loss = 0
        for b in range(o.shape[0]):
            heights_for_sample = pred_heights[batch_indices == b]
            if heights_for_sample.numel() > 0:
                total_height_loss += torch.abs(heights_for_sample.sum() - 1)

        return {'loss_height_constraint': total_height_loss / o.shape[0]}

    def _get_src_p_idx(self,i):
        b=torch.cat([torch.full_like(s,k) for k,(s,_) in enumerate(i)]); s=torch.cat([s for s,_ in i]); return b,s

    def forward(self,o,t):
        i=self.m(o,t);
        losses = {}
        losses.update(self.loss_match(o,t,i))
        losses.update(self.loss_unmatch(o,t,i))
        losses.update(self.loss_height(o,t,i))
        return {ln: l for ln,l in losses.items() if self.w[ln]>0}

print("✅ Matcher and loss functions defined.")

✅ Matcher and loss functions defined.


In [9]:
#@title 9. Define Helper and Utility Functions

def collate_fn(batch):
    images, targets = zip(*batch)
    return torch.stack(images), list(targets)

def load_pretrained_encoder_weights(model, path):
    print(f"--> Loading pre-trained weights from {path}")
    pre_dict = torch.load(path)
    model_dict = model.state_dict()
    enc_dict = {k.replace('module.',''):v for k,v in pre_dict.items() if any(x in k for x in ['e1','e2','e3','mid','start'])}
    enc_dict = {'encoder.'+k:v for k,v in enc_dict.items()}
    model_dict.update(enc_dict)
    model.load_state_dict(model_dict, strict=False)
    print(f"✅ Loaded {len(enc_dict)} pre-trained layers.")
    return model

print("✅ Helper functions defined.")

✅ Helper functions defined.


In [10]:
#@title 10. Define Pre-training Stage Function (W&B Sweep) (Corrected)
from tqdm import tqdm

# A global variable to track the best loss across all sweep runs
best_pretrain_loss = float('inf')

def train_autoencoder_sweep():
    global best_pretrain_loss, config
    with wandb.init() as run:
        sweep_cfg = wandb.config
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        # --- CORRECTED: Use the new UNet1D model ---
        model = UNet1D(in_channels=sweep_cfg.in_channels, activation=sweep_cfg.act_name).to(device)

        criterion = nn.MSELoss()
        optimizer = getattr(torch.optim, sweep_cfg.optimizer)(model.parameters(), lr=sweep_cfg.lr)
        train_loader = data.DataLoader(AutoencoderDataset(config), batch_size=sweep_cfg.batch_size, shuffle=True)

        print(f"--- Starting W&B Run with config: {dict(sweep_cfg)} ---")
        for epoch in range(sweep_cfg.epochs):
            model.train()
            total_loss = 0.0
            for img, tgt in train_loader:
                img, tgt = img.to(device), tgt.to(device)
                optimizer.zero_grad()
                output = model(img)
                loss = criterion(output, tgt)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()

            epoch_loss = total_loss / len(train_loader)
            wandb.log({"epoch": epoch, "loss": epoch_loss})

            if epoch_loss < best_pretrain_loss:
                best_pretrain_loss = epoch_loss
                print(f"    *** New best model found! Loss: {best_pretrain_loss:.6f} (Epoch {epoch+1}) ***")
                torch.save(model.state_dict(), config['paths']['pretrained_encoder_path'])
                wandb.summary["best_loss"] = best_pretrain_loss

print("✅ Pre-training (sweep) function defined with 1D U-Net.")

✅ Pre-training (sweep) function defined with 1D U-Net.


In [11]:
#@title 11. Define Fine-tuning Stage Function

def run_finetuning(config):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    ft_params = config['finetuning']
    loader = data.DataLoader(BoundaryDataset(config, seed=42), batch_size=ft_params['batch_size'], shuffle=True, collate_fn=collate_fn)
    model = W2WTransformerModel(config).to(device)
    model = load_pretrained_encoder_weights(model, config['paths']['pretrained_encoder_path'])
    criterion = SetCriterion(config).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=ft_params['learning_rate'])

    for epoch in range(ft_params['epochs']):
        model.train()
        total_loss = 0
        for images, targets in tqdm(loader, desc=f'Epoch {epoch+1}/{ft_params["epochs"]}'):
            images, targets = images.to(device), [{k: v.to(device) for k, v in t.items()} for t in targets]
            loss_dict = criterion(model(images), targets)
            losses = sum(loss_dict[k] * criterion.w[k] for k in loss_dict.keys())
            optimizer.zero_grad(); losses.backward(); optimizer.step()
            total_loss += losses.item(); wandb.log({'finetune_batch_loss': losses.item()})

        avg_loss = total_loss / len(loader)
        print(f'Epoch {epoch+1} Average Loss: {avg_loss:.4f}')
        wandb.log({'finetune_epoch_loss': avg_loss, 'epoch': epoch})

    torch.save(model.state_dict(), config['paths']['final_model_path'])
    print(f"✅ Final model saved to {config['paths']['final_model_path']}")

print("✅ Fine-tuning function defined.")

✅ Fine-tuning function defined.


In [12]:
#@title 12. Define Inference and Plotting Functions
import matplotlib.pyplot as plt
import matplotlib.patches as patches

def plot_well_correlation(well1, well2, layers1, layers2, matrix, threshold, path):
    fig, ax = plt.subplots(figsize=(10, 12)); plt.style.use('seaborn-whitegrid')
    if not layers1 or not layers2: print('Warning: One or both wells have no layers to plot.'); return
    max_depth = max(layers1[-1]['bottom'], layers2[-1]['bottom']) if layers1 and layers2 else 1000
    ax.set_ylim(max_depth + 50, -50); ax.set_xlim(-0.5, 2.5)
    n1 = len(set(l['Group'] for l in layers1)); n2 = len(set(l['Group'] for l in layers2))
    for l in layers1: ax.add_patch(patches.Rectangle((0, l['Top']), 1, l['Height'], ec='k', fc=plt.cm.viridis(l['Group']/(n1 if n1>0 else 1)), alpha=0.6))
    for l in layers2: ax.add_patch(patches.Rectangle((1.5, l['Top']), 1, l['Height'], ec='k', fc=plt.cm.viridis(l['Group']/(n2 if n2>0 else 1)), alpha=0.6))
    for i, row in enumerate(matrix):
        for j, sim in enumerate(row):
            if sim >= threshold: ax.add_patch(patches.Polygon([[1,layers1[i]['Top']],[1,layers1[i]['bottom']],[1.5,layers2[j]['bottom']],[1.5,layers2[j]['Top']]], fc=plt.cm.Greens(sim), alpha=0.5))
    ax.set_xticks([0.5, 2]); ax.set_xticklabels([well1, well2], fontsize=14); ax.set_ylabel('Depth', fontsize=12)
    ax.set_title('Well to Well Correlation', fontsize=16); plt.savefig(path); plt.close()
    print(f'--> Correlation plot saved to {path}')

def run_correlation(config):
    inf, p = config['inference'], config['paths']
    full_data = pd.read_csv(p['processed_csv_path'], delimiter=';')
    ref_df, woi_df = full_data[full_data['WELL'] == inf['reference_well']], full_data[full_data['WELL'] == inf['well_of_interest']]
    if ref_df.empty or woi_df.empty: print(f"Error: One/both wells not found: '{inf['reference_well']}', '{inf['well_of_interest']}'."); return

    with open(p['label_encoder_path']) as f: le = json.load(f)
    def get_true_layers(df):
        df = df.copy().reset_index(drop=True)
        df['group_id'] = df['GROUP'].astype(str).map(le).fillna(-1).astype(int)
        b = np.where(df['group_id'].iloc[:-1].values != df['group_id'].iloc[1:].values)[0] + 1
        indices = np.concatenate(([0], b, [len(df)]))
        layers = []
        for i in range(len(indices) - 1):
            s, e = indices[i], indices[i+1]
            layers.append({'Top':df['DEPTH_MD'].iloc[s],'bottom':df['DEPTH_MD'].iloc[e-1],'Height':df['DEPTH_MD'].iloc[e-1]-df['DEPTH_MD'].iloc[s],'Group':df['group_id'].iloc[s]})
        return layers

    ref_layers, woi_layers = get_true_layers(ref_df), get_true_layers(woi_df)
    sim_matrix = np.zeros((len(ref_layers), len(woi_layers)))
    for i,l1 in enumerate(ref_layers):
        for j,l2 in enumerate(woi_layers):
            if l1['Group'] == l2['Group'] and l1['Group'] != -1: sim_matrix[i,j] = np.random.uniform(0.8, 0.95)
            else: sim_matrix[i,j] = np.random.uniform(0.1, 0.4)

    print('--> MOCK INFERENCE: Using ground truth layers for visualization.')
    output_path = 'well_correlation_plot.png'
    plot_well_correlation(inf['reference_well'], inf['well_of_interest'], ref_layers, woi_layers, sim_matrix, inf['correlation_threshold'], output_path)
    wandb.log({"well_correlation_plot": wandb.Image(output_path)})

print("✅ Inference and plotting functions defined.")

✅ Inference and plotting functions defined.


In [13]:
#@title 13. 🚀 Run the Full Pipeline

# --- STAGE 0: DATA PREPARATION ---
if config["run_data_preparation"]:
    run_data_preparation(config)
    print("\n--- STAGE 0 COMPLETE ---\n")

# --- STAGE 1: PRE-TRAINING (W&B SWEEP) ---
if config.get('run_pretraining', False):
    print("\n--- LAUNCHING PIPELINE 1: AUTOENCODER PRE-TRAINING (W&B SWEEP) ---")
    sweep_id = wandb.sweep(config['pretraining_sweep'], project=config['wandb']['project'], entity=config['wandb'].get('entity'))
    wandb.agent(sweep_id, function=train_autoencoder_sweep, count=config['wandb']['sweep_count'])
    print(f"\n🏆 Sweep finished. Best pre-trained model saved to {config['paths']['pretrained_encoder_path']}")
    print("\n--- STAGE 1 COMPLETE ---\n")

# --- STAGE 2: FINE-TUNING ---
if config.get('run_finetuning', False):
    print("\n--- LAUNCHING PIPELINE 2: FINE-TUNING ---")
    with wandb.init(project=config['wandb']['project'], entity=config['wandb'].get('entity'), job_type='fine-tuning', config=config) as run:
        print(f"--> W&B Run started. View at: {run.get_url()}")
        run_finetuning(config)
        artifact = wandb.Artifact("boundary-detector-model", type="model", description="Final fine-tuned W2W Transformer model")
        artifact.add_file(config['paths']['final_model_path'])
        run.log_artifact(artifact)
        print("✅ Final model logged as a W&B Artifact.")
    print("\n--- STAGE 2 COMPLETE ---\n")

# --- STAGE 3: INFERENCE ---
if config.get('run_inference', False):
    print("\n--- LAUNCHING PIPELINE 3: WELL-TO-WELL INFERENCE ---")
    with wandb.init(project=config['wandb']['project'], entity=config['wandb'].get('entity'), job_type='inference', config=config) as run:
        print(f"--> W&B Run started. View at: {run.get_url()}")
        run_correlation(config)
    print("\n--- STAGE 3 COMPLETE ---\n")

print("\n" + "="*60)
print("✅✅✅ All Requested Pipeline Stages are Complete! ✅✅✅")
print("You can view all your results, models, and charts in your Weights & Biases project.")
print("="*60)

--- LAUNCHING PIPELINE 0: DATA PREPARATION ---
--> Searching for .las files in 'data/raw_las_files/'...
--> Found 118 .las files. Reading now...
--> Saved combined data to 'data/train.csv'

--- Available Well Names for Inference ---
- 35/4-1
- 34/8-7R
- 16/11-1S T3
- 7/1-2 S
- 34/3-2 S
- 31/2-10
- 25/7-2
- 25/11-24 Jakob South
- 25/10-9 Aegis
- 34/10-33
- 16/7-6
- 34/3-3 A
- 31/2-8
- 31/4-5
- 35/8-6 S
- 34/4-10 R
- 16/2-6 Johan Sverdrup
- 31/2-1
- 25/6-2  Delta-Beta
- 31/2-9
- 35/11-11
- 34/3-1 A
- 35/9-2
- 31/2-19 S
- 31/6-8
- 25/8-5 S  Jotun
- 16/10-2 Delta
- 34/10-21
- 25/3-1
- 16/8-1
- 31/3-4
- 35/11-13
- 34/10-19
- 25/10-10  Balder Triassic
- 36/7-3
- 15/9-23 Skardkollen
- 30/3-3
- 34/8-1
- 34/11-2 S
- 16/7-5
- 34/7-13
- 31/5-4 S
- 16/2-7 Johan Sverdrup Appr
- 25/2-13 T4
- 16/10-1 Alpha
- 35/11-15 S
- 32/2-1
- 16/4-1
- 35/3-7 S
- 34/8-3
- 35/11-10
- 34/11-1
- 35/12-1
- 25/11-5 Balder Appr
- 35/11-7
- 16/1-2  Ivar Aasen Appr
- 33/6-3 S
- 35/11-12
- 15/9-14
- 31/2-7
- 35/11-1
- 34/7

wandb: Agent Starting Run: atzyttuu with config:
wandb: 	act_name: relu
wandb: 	batch_size: 32
wandb: 	epochs: 25
wandb: 	in_channels: 25
wandb: 	lr: 0.0001
wandb: 	optimizer: RMSprop
wandb: Currently logged in as: sahilpareek203 (sahilpareek203-amrita-vishwa-vidyapeetham) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


--- Starting W&B Run with config: {'act_name': 'relu', 'batch_size': 32, 'epochs': 25, 'in_channels': 25, 'lr': 0.0001, 'optimizer': 'RMSprop'} ---
    *** New best model found! Loss: 0.821698 (Epoch 1) ***
    *** New best model found! Loss: 0.685048 (Epoch 2) ***
    *** New best model found! Loss: 0.628560 (Epoch 3) ***
    *** New best model found! Loss: 0.591594 (Epoch 4) ***
    *** New best model found! Loss: 0.560599 (Epoch 5) ***
    *** New best model found! Loss: 0.537167 (Epoch 6) ***
    *** New best model found! Loss: 0.514409 (Epoch 7) ***
    *** New best model found! Loss: 0.492698 (Epoch 8) ***
    *** New best model found! Loss: 0.476900 (Epoch 9) ***
    *** New best model found! Loss: 0.465546 (Epoch 10) ***
    *** New best model found! Loss: 0.449828 (Epoch 11) ***
    *** New best model found! Loss: 0.438199 (Epoch 12) ***
    *** New best model found! Loss: 0.428672 (Epoch 13) ***
    *** New best model found! Loss: 0.418053 (Epoch 14) ***
    *** New best mode

epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
loss,█▆▅▅▄▄▄▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▂▁
best_loss,0.3431
epoch,24
loss,0.3431


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: xg2aixgc with config:
wandb: 	act_name: prelu
wandb: 	batch_size: 16
wandb: 	epochs: 25
wandb: 	in_channels: 25
wandb: 	lr: 0.0001
wandb: 	optimizer: AdamW


...

Epoch 200 Average Loss: 0.2008
✅ Final model saved to trained_models/boundary_detector/final_model.pt
✅ Final model logged as a W&B Artifact.


epoch,▁▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
finetune_batch_loss,█▇▆▅▅▄▅▅▄▆▅▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁
finetune_epoch_loss,██▆▆▆▆▅▅▅▅▅▅█▆▅▅▅▄▄▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▁▁▁▁
epoch,199
finetune_batch_loss,0.20078
finetune_epoch_loss,0.20082



--- STAGE 2 COMPLETE ---


--- LAUNCHING PIPELINE 3: WELL-TO-WELL INFERENCE ---


--> W&B Run started. View at: https://wandb.ai/sahilpareek203-amrita-vishwa-vidyapeetham/W2W_Matcher_Pipeline_Notebook/runs/cgkgh9tl
Error: One/both wells not found: '15_9-13 Sleipner East Appr', '16/1-2  Ivar Aasen Appr'.



--- STAGE 3 COMPLETE ---


✅✅✅ All Requested Pipeline Stages are Complete! ✅✅✅
You can view all your results, models, and charts in your Weights & Biases project.
